In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn.model_selection import TimeSeriesSplit
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, BatchNormalization, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, Callback
from tensorflow.keras.regularizers import l2
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

df = pd.read_csv("data/ilapak3/train.csv")
df['times'] = pd.to_datetime(df['times'])
df = df.sort_values(by='times').reset_index(drop=True)

df.head()

# Feature Engineering & Scalling

In [ ]:
# Times
df["day"] = df["times"].dt.day
df["hour"] = df["times"].dt.hour
df["minute"] = df["times"].dt.minute

# Diff of sealing
df["diff_sealing_vertical"] = df["Suhu Sealing Vertical Atas (oC)"] - df["Suhu Sealing Vertikal Bawah (oC)"]
df["diff_sealing_horizontal"] = df["Suhu Sealing Horizontal Depan/Kanan (oC)"] - df["Suhu Sealing Horizontal Belakang/Kiri (oC )"]

# Diff of output reject
df["diff_output"] = df["Counter Output (pack)"] - df["Counter Reject (pack)"]

# Diff of output before
df["delta_output"] = df["Counter Output (pack)"].diff(periods=5)
df["delta_reject"] = df["Counter Reject (pack)"].diff(periods=5)

df.fillna(0, inplace=True)

In [ ]:
df.columns

## Scale for Continuous

In [ ]:
continuous_cols = [
    "Suhu Sealing Vertikal Bawah (oC)",
    "Suhu Sealing Vertical Atas (oC)",
    "Suhu Sealing Horizontal Depan/Kanan (oC)",
    "Suhu Sealing Horizontal Belakang/Kiri (oC )",
    "diff_sealing_vertical",
    "diff_sealing_horizontal",
    "diff_output",
    "delta_output",
    "delta_reject",
    "Counter Output (pack)",
    "Counter Reject (pack)",
    "Availability(%)",
    "Performance(%)",
    "Quality(%)",
    "OEE(%)",
    "Speed(rpm)",
    "day",
    "hour",
    "minute",
    'Downtime_sec',
    'Output Time_sec', 'Total Time_sec'
]

categorical_cols = [
    "Status",
    "Shift",
    'Jaws Position',
       'Doser Drive Enable', 'Sealing Enable', 'Machine Alarm'

]

In [ ]:
X = df.drop(columns=["Condition"])
y = df["Condition"]

preprocessor = ColumnTransformer(
    [
        ("num", MinMaxScaler(), continuous_cols),
        ("cat", "passthrough", categorical_cols),
    ]
)

X_all = preprocessor.fit_transform(X)
y_all = y

# Apply SMOTE
# smote = SMOTE(random_state=42)
# X_resampled, y_resampled = smote.fit_resample(X_all, y_all)

# Sliding Window Preparation

In [ ]:
def create_sliding_windows(X, y, window_size):
    X_seq, y_seq = [], []
    for i in range(len(X) - window_size + 1):
        X_seq.append(X[i:i+window_size])
        y_seq.append(y[i+window_size-1])
    return np.array(X_seq), np.array(y_seq)

window_size = 10
X_seq, y_seq = create_sliding_windows(X_all, y_all, window_size)

print(f"X Sequence Shape: {X_seq.shape}")
print(f"y Sequence Shape: {y_seq.shape}")

# Splitting Dataset

In [ ]:
tscv = TimeSeriesSplit(n_splits=10)
for train_index, val_index in tscv.split(X_seq):
    X_train, X_val = X_seq[train_index], X_seq[val_index]
    y_train, y_val = y_seq[train_index], y_seq[val_index]

print(f"X Train Shape: {X_train.shape}")
print(f"y Train Shape: {y_train.shape}")
print(f"X Val Shape: {X_val.shape}")
print(f"y Val Shape: {y_val.shape}")

# Build & Train BiLSTM

## Define Callback

In [ ]:
class PlotTraining(Callback):
    def __init__(self, interval=10):
        self.interval = interval
        self.history = {'loss': [], 'val_loss': [], 'accuracy': [], 'val_accuracy': []}

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        for key in self.history:
            if key in logs:
                self.history[key].append(logs[key])
        
        if (epoch + 1) % self.interval == 0:
            self.plot_metrics(epoch)

    def plot_metrics(self, epoch):
        epochs = range(1, epoch + 2)
        plt.figure(figsize=(12, 5))

        # Loss
        plt.subplot(1, 2, 1)
        plt.plot(epochs, self.history['loss'], label='Train Loss')
        plt.plot(epochs, self.history['val_loss'], label='Val Loss')
        plt.title('Loss')
        plt.xlabel('Epoch')
        plt.legend()

        # Accuracy
        plt.subplot(1, 2, 2)
        plt.plot(epochs, self.history['accuracy'], label='Train Acc')
        plt.plot(epochs, self.history['val_accuracy'], label='Val Acc')
        plt.title('Accuracy')
        plt.xlabel('Epoch')
        plt.legend()

        plt.tight_layout()
        plt.show()

## Config

In [ ]:
NUM_CLASSES = 3
BATCH_SIZE = 64
INPUT_SHAPE = (X_train.shape[1], X_train.shape[2])

In [ ]:
INPUT_SHAPE

# Data Pipeline

In [ ]:
def create_dataset(X, y, batch_size=32, shuffle=False):
    ds = tf.data.Dataset.from_tensor_slices((X, y))
    if shuffle:
        ds = ds.shuffle(buffer_size=10000)
    ds = ds.batch(batch_size).cache().prefetch(tf.data.AUTOTUNE)
    return ds

train_ds = create_dataset(X_train, y_train, batch_size=BATCH_SIZE, shuffle=True)
val_ds = create_dataset(X_val, y_val, batch_size=BATCH_SIZE)

## Arhictecture

In [ ]:
model = Sequential([
    Conv1D(filters=16, kernel_size=3, activation='relu', padding='same',
           kernel_regularizer=l2(1e-2), input_shape=INPUT_SHAPE),
    BatchNormalization(),

    Conv1D(filters=32, kernel_size=3, activation='relu', padding='same',
           kernel_regularizer=l2(1e-2
           )),
    BatchNormalization(),

    MaxPooling1D(pool_size=2),

    Bidirectional(LSTM(128, return_sequences=False, dropout=0.4,
                       kernel_regularizer=l2(1e-4))),
    BatchNormalization(),

    Dense(64, activation='relu', kernel_regularizer=l2(1e-4)),
    Dropout(0.4),

    Dense(32, activation='relu', kernel_regularizer=l2(1e-4)),
    BatchNormalization(),

    Dense(NUM_CLASSES, activation='softmax')
])

model.summary()

## Training

In [ ]:
def focal_loss(alpha=0.25, gamma=2.0):
    def focal_loss_fixed(y_true, y_pred):
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
        
        # Convert to one-hot
        y_true = tf.cast(y_true, tf.int32)
        y_true = tf.one_hot(y_true, depth=3)
        y_true = tf.cast(y_true, tf.float32)
        
        # Calculate focal loss
        ce = -y_true * tf.math.log(y_pred)
        weight = alpha * y_true * tf.pow((1 - y_pred), gamma)
        fl = weight * ce
        reduced_fl = tf.reduce_sum(fl, axis=1)
        return tf.reduce_mean(reduced_fl)
    
    return focal_loss_fixed

optimizer = AdamW(learning_rate=0.003)
callbacks = [
    EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1),
    PlotTraining(interval=10)
]

# Compile
model.compile(
    optimizer=optimizer,
    loss=focal_loss(alpha=0.25, gamma=3.0),
    metrics=['accuracy']
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=100,
    callbacks=callbacks,
    verbose=1
)

# Evaluation

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

In [ ]:
y_pred_prob = model.predict(X_val)
y_pred = y_pred_prob.argmax(axis=1)

print(classification_report(y_val, y_pred, target_names=["normal", "warning", "leak"]))

cm = confusion_matrix(y_val, y_pred)
cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

fig, axs = plt.subplots(1, 2, figsize=(14, 5))

sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["normal", "warning", "leak"], yticklabels=["normal", "warning", "leak"], ax=axs[0])
axs[0].set_title('Confusion Matrix (Counts)')
axs[0].set_xlabel('Predicted')
axs[0].set_ylabel('Actual')

sns.heatmap(cm_percent * 100, annot=True, fmt='.1f', cmap='Greens', xticklabels=["normal", "warning", "leak"], yticklabels=["normal", "warning", "leak"], ax=axs[1])
axs[1].set_title('Confusion Matrix (%)')
axs[1].set_xlabel('Predicted')
axs[1].set_ylabel('Actual')

plt.show()

# Prediction Pipeline

In [ ]:
def predict_condition(new_df, feature_cols_cat, feature_cols_cont, scaler, model, window_size=5):
    new_df = new_df.sort_values(by='times')
    assert len(new_df) >= window_size, "Butuh minimal window_size data"

    X_cat = new_df[feature_cols_cat].values
    X_cont = scaler.transform(new_df[feature_cols_cont])
    X_input = np.hstack([X_cat, X_cont])[-window_size:]
    X_input = X_input.reshape(1, window_size, -1)

    pred_probs = model.predict(X_input)
    pred_class = pred_probs.argmax(axis=1)[0]
    return pred_class, pred_probs[0]

# Save Model and Scaler

In [ ]:
# from tensorflow.keras.models import save_model
# import joblib

# save_model(model, "bilstm_model.h5")
# joblib.dump(scaler, "scaler.pkl")